# Smart AI Study Assistant — Notebook Runner (with 5 required features)

This notebook demonstrates **five key GenAI Agent features** using a fully offline, self-contained multi-agent system:

### **1. Agent Powered by an LLM**
A mocked LLM agent (`MockLLMAgent`) performs:
- explanations
- quizzes
- summaries
- prerequisite and key concept guidance

### **2. Parallel Agents**
Using `concurrent.futures.ThreadPoolExecutor`, two agents run in parallel:
- `ContentSearchAgent` (resources)
- `FlashcardAgent` (flashcards)

### **3. Sequential Agents**
After the parallel work:
- quizzes
- informative queries  
are generated sequentially by the LLM agent.

### **4. Sessions & Memory**
`InMemorySessionService` stores:
- session state
- topics
- weekly hours
- last generated study plan

This enables personalized learning and state tracking.

### **5. Observability & Long-Running Operations**
Two optional but highly valuable features are included:

#### ✔ Logging (Observability)
All agents log:
- start/finish messages  
- progress  
- warnings  
- pause/resume events  

#### ✔ Long-running Agent Simulation (Pause & Resume)
A `LongRunningAgent` simulates a task that:
- progresses in steps  
- can be paused  
- can be resumed  
- logs its state  

---

All external integrations (real LLMs, search APIs) are mocked so the notebook runs fully offline and reproducibly.

This notebook satisfies more than the **minimum three** features required for the GenAI Capstone.


### Abstract
This notebook implements a multi-agent AI Study Assistant using a mocked LLM, parallel agents, sequential orchestration, session memory, observability via logging, and a long-running pause/resume agent. It satisfies 5 required agent features and runs fully offline.


## 1. Architecture Diagram

<pre>
User
  ↓
StudyCoordinator
    ├── LLM Agent (Mock/Gemini)
    ├── ContentSearchAgent
    ├── FlashcardAgent
    └── Session Service
</pre>


## 2. Import Components

Now, import the specific components you'll need. This keeps your code organized and ensures we have access to the necessary building blocks.

In [2]:
import time
import uuid
import concurrent.futures
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

print("✅ Components imported successfully.")

✅ Components imported successfully.


## 3. Observability: Logging

We add Python's built-in logging to track agent activity:
- when agents start
- when they finish
- errors
- coordinator steps

This satisfies the Observability feature.


In [3]:
import logging

# Configure global logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(asctime)s — %(message)s',
    datefmt='%H:%M:%S'
)

log = logging.getLogger(__name__)

log.info("Logging initialized.")

[INFO] 15:39:39 — Logging initialized.


## 4. Core Data Models

We start by defining a few simple data classes that will be used throughout the system:

- `LearningObjective`: represents a topic, its difficulty, and its prerequisites.
- `Resource`: represents a learning resource (e.g., video, article) with a relevance score.
- `Flashcard`: represents a basic flashcard with a question, answer, and ease factor (for spaced repetition later).

These classes make the rest of the code more readable and structured.


In [4]:
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

# -----------------------------
# Data classes
# -----------------------------

@dataclass
class LearningObjective:
    topic: str
    difficulty: float
    prerequisites: List[str]

@dataclass
class Resource:
    title: str
    url: str
    summary: str
    relevance: float

@dataclass
class Flashcard:
    question: str
    answer: str
    ease: float = 2.5  # initial ease factor (used in spaced repetition systems)


## 5. Session & Memory Service

To simulate "memory" and personalization, we use a simple in-memory session service.

- Each session is identified by a `session_id` (UUID).
- We store metadata (like preferred format) and a `state` dictionary.
- This allows the system to remember user topics, plans, and preferences across calls.


In [5]:
import time
import uuid

# -----------------------------
# In-memory session service
# -----------------------------

class InMemorySessionService:
    """Simple session store keyed by session_id."""
    def __init__(self):
        self.sessions: Dict[str, Dict[str, Any]] = {}

    def create_session(self, user_id: str, metadata: Optional[Dict[str, Any]] = None) -> str:
        sid = str(uuid.uuid4())
        self.sessions[sid] = {
            "user_id": user_id,
            "created_at": time.time(),
            "metadata": metadata or {},
            "state": {}
        }
        return sid

    def get(self, session_id: str) -> Dict[str, Any]:
        return self.sessions.get(session_id, {})

    def set_state(self, session_id: str, key: str, value: Any):
        if session_id not in self.sessions:
            raise KeyError("session not found")
        self.sessions[session_id]["state"][key] = value

    def get_state(self, session_id: str, key: str, default=None):
        return self.sessions.get(session_id, {}).get("state", {}).get(key, default)

print("✅ InMemorySessionService defined.")


✅ InMemorySessionService defined.


## 6. Mock LLM Agent (Agent powered by an LLM)

In a real system, this component would call an external LLM (e.g., Gemini, GPT, etc.).
For this notebook, we use a **mocked LLM agent** that:

- Summarizes text.
- Explains a topic at a given level.
- Generates simple quiz questions.
- Answers predefined informative queries (prerequisites, key concepts, practice problems).

This keeps the notebook fully offline and reproducible.


In [6]:
# -----------------------------
# Mock LLM Agent (feature: Agent powered by an LLM)
# -----------------------------

class MockLLMAgent:
    """A mocked LLM agent that 'generates' summaries, explanations, quiz questions and answers queries.
    In a real system this would call an LLM API with prompt engineering and safety checks.
    """
    def summarize(self, text: str, max_length: int = 60) -> str:
        # light-weight fake summarization
        return (text[:max_length] + '...') if len(text) > max_length else text

    def explain(self, topic: str, level: str = 'intro') -> str:
        return (
            f"Explanation ({level}) for {topic}: "
            f"This is a concise, learner-friendly explanation covering the main ideas, examples, and intuition."
        )

    def generate_quiz(self, topic: str, n_questions: int = 3) -> List[Dict[str, Any]]:
        qs = []
        for i in range(1, n_questions + 1):
            qs.append({
                "q": f"Sample question {i} about {topic}",
                "a": f"Short model answer for question {i} on {topic}."
            })
        return qs

    def answer_query(self, topic: str, query_type: str) -> str:
        """Return short, informative answers for a set of predefined query types."""
        if query_type == "prerequisites":
            return (
                f"Prerequisites for {topic}: basic Python, linear algebra (vectors/matrices), "
                "probability fundamentals, and comfort with reading technical blogs."
            )
        if query_type == "key_concepts":
            return (
                f"Key concepts for {topic}: core definitions, main algorithms, complexity trade-offs, "
                "common failure modes, and real-world examples."
            )
        if query_type == "practice_problems":
            return (
                f"Practice ideas for {topic}: implement a small example from scratch, solve 3 conceptual problems, "
                "and reproduce one result from a short tutorial."
            )
        # default fallback
        return f"Short guidance for {topic} regarding '{query_type}': try learning by building a small project."

print("✅ MockLLMAgent defined.")


✅ MockLLMAgent defined.


## 7. Other Agents: Content Search & Flashcards

We now define two additional agents:

- `ContentSearchAgent`: simulates a search over learning resources.
  - For this demo, it always returns a YouTube playlist (Dr Abhishek's AI Agent Intensive)
    and a placeholder "deep dive" resource.
- `FlashcardAgent`: uses the LLM agent to generate basic flashcards for a topic.

These will be used as **parallel agents** that run concurrently.


In [7]:
# -----------------------------
# Other agents (search & flashcard generator)
# -----------------------------

class ContentSearchAgent:
    def run(self, topic: str) -> List[Resource]:
        log.info(f"[SearchAgent] Starting search for: {topic}")
        time.sleep(0.4)  # simulate latency
        log.info(f"[SearchAgent] Finished search for: {topic}")
        
        playlist = "https://www.youtube.com/@KulDeepLearns"
        return [
            Resource(
                title=f"{topic} — Kuldeep's Playlist",
                url=playlist,
                summary=f"A curated playlist covering {topic}.",
                relevance=0.95
            ),
            Resource(
                title=f"{topic} — Deep dive article",
                url="https://example.com/deep",
                summary=f"In-depth notes for {topic}.",
                relevance=0.75
            )
        ]

class FlashcardAgent:
    def __init__(self, llm_agent: MockLLMAgent):
        self.llm = llm_agent

    def run(self, topic: str, n_cards: int = 5) -> List[Flashcard]:
        log.info(f"[FlashcardAgent] Generating flashcards for: {topic}")
        cards = []
        for i in range(n_cards):
            q = f"What is key idea {i+1} in {topic}?"
            a = self.llm.explain(topic, level='short')
            cards.append(Flashcard(question=q, answer=a))
        log.info(f"[FlashcardAgent] Finished flashcards for: {topic}")
        return cards


print("✅ ContentSearchAgent and FlashcardAgent defined.")


✅ ContentSearchAgent and FlashcardAgent defined.


## 8. StudyCoordinator: Orchestrating Parallel Agents & Memory

The `StudyCoordinator` ties everything together:

- Uses the `InMemorySessionService` to store:
  - topics
  - weekly study hours
  - last generated plan
- Runs **content search** and **flashcard generation** in parallel with `ThreadPoolExecutor`.
- Uses the LLM agent to:
  - generate a short quiz per topic
  - answer "informative queries" (prerequisites, key concepts, practice problems)

This class demonstrates:
- An agent powered by an LLM (mock),
- Parallel agents (search + flashcards),
- Session-based memory.


In [8]:
# -----------------------------
# Coordinator demonstrating parallel agents + sessions + informative queries
# -----------------------------

class StudyCoordinator:
    def __init__(self, session_service: InMemorySessionService):
        self.llm = MockLLMAgent()
        self.search_agent = ContentSearchAgent()
        self.flashcard_agent = FlashcardAgent(self.llm)
        self.sessions = session_service

    def create_or_get_session(self, user_id: str, metadata: Optional[Dict[str, Any]] = None) -> str:
        # For this demo, we always create a fresh session
        return self.sessions.create_session(user_id, metadata)

    def plan_for(self, session_id: str, topics: List[str], weekly_hours: int = 5) -> Dict[str, Any]:
        # Save requested topics to session state
        self.sessions.set_state(session_id, 'topics', topics)
        self.sessions.set_state(session_id, 'weekly_hours', weekly_hours)

        # Prepare results container
        results = {"resources": {}, "flashcards": {}, "quizzes": {}, "informative_queries": {}}

        # Run content search and flashcard generation in parallel for each topic
        with concurrent.futures.ThreadPoolExecutor(max_workers=6) as ex:
            # schedule futures
            future_to_topic_search = {ex.submit(self.search_agent.run, t): t for t in topics}
            future_to_topic_flash = {ex.submit(self.flashcard_agent.run, t, 4): t for t in topics}
            
            # collect search results
            for fut in concurrent.futures.as_completed(list(future_to_topic_search.keys())):
                topic = future_to_topic_search[fut]
                try:
                    resources = fut.result()
                except Exception as e:
                    print(f"[WARN] Search agent failed for topic '{topic}': {e}")
                    resources = []
                results['resources'][topic] = resources

            # collect flashcards
            for fut in concurrent.futures.as_completed(list(future_to_topic_flash.keys())):
                topic = future_to_topic_flash[fut]
                try:
                    cards = fut.result()
                except Exception as e:
                    print(f"[WARN] Flashcard agent failed for topic '{topic}': {e}")
                    cards = []
                results['flashcards'][topic] = cards

        # Use LLM agent sequentially to create a short quiz per topic
        for topic in topics:
            quiz = self.llm.generate_quiz(topic, n_questions=3)
            results['quizzes'][topic] = quiz

        # Add informative queries (prerequisites, key concepts, practice problems)
        query_types = ["prerequisites", "key_concepts", "practice_problems"]
        for topic in topics:
            answers = {}
            for qt in query_types:
                ans = self.llm.answer_query(topic, qt)
                answers[qt] = ans
            results['informative_queries'][topic] = answers

        # Save summary into session for later retrieval
        self.sessions.set_state(session_id, 'last_plan', results)
        return results

print("✅ StudyCoordinator defined.")

✅ StudyCoordinator defined.


## 9. Demo Run: Smart AI Study Assistant in Action

Finally, we create:

- A new session for a dummy user,
- A list of 5 topics (representing a 5-day AI Agent Intensive),
- A study plan using `StudyCoordinator.plan_for`.

We then print a readable summary showing:

- Resources per topic (from the search agent),
- Number of flashcards generated,
- Quiz questions,
- Informative LLM answers (prerequisites, key concepts, practice problems).


In [9]:
# Initialize services and coordinator
session_service = InMemorySessionService()
coord = StudyCoordinator(session_service)

# Create a new session for a demo user
sid = coord.create_or_get_session(
    user_id='user_123',
    metadata={'preferred_format': 'videos+problems'}
)
print("Created session id:", sid)

# Use 5 topics representing the 5-day AI Agent Intensive by Kuldeep
topics = [
    'Day 1 — Foundations of AI Agents',
    'Day 2 — Tools & Integrations',
    'Day 3 — Planning & Orchestration',
    'Day 4 — Budgeting & Optimization',
    'Day 5 — Deployment & Observability'
]

# Generate a plan
plan_result = coord.plan_for(sid, topics, weekly_hours=10)

print('\n--- Plan Summary (topics) ---')
for t in topics:
    res = plan_result['resources'].get(t, [])
    print(f"\nTopic: {t}")
    
    print('  Resources:')
    for r in res:
        print(f"    - {r.title} | {r.url} | {r.summary[:80]}...")
    
    cards = plan_result['flashcards'].get(t, [])
    print(f'  Flashcards: {len(cards)} created')
    
    quiz = plan_result['quizzes'].get(t, [])
    print(f'  Quiz: {len(quiz)} questions')
    
    info = plan_result['informative_queries'].get(t, {})
    print('  Informative queries:')
    for qtype, answer in info.items():
        print(f'    - {qtype}: {answer}')

print('\nSession stored state keys:', list(session_service.get(sid).get('state', {}).keys()))


[INFO] 15:40:32 — [SearchAgent] Starting search for: Day 1 — Foundations of AI Agents
[INFO] 15:40:32 — [SearchAgent] Starting search for: Day 2 — Tools & Integrations
[INFO] 15:40:32 — [SearchAgent] Starting search for: Day 3 — Planning & Orchestration
[INFO] 15:40:32 — [SearchAgent] Starting search for: Day 4 — Budgeting & Optimization
[INFO] 15:40:32 — [SearchAgent] Starting search for: Day 5 — Deployment & Observability
[INFO] 15:40:32 — [FlashcardAgent] Generating flashcards for: Day 1 — Foundations of AI Agents
[INFO] 15:40:32 — [FlashcardAgent] Finished flashcards for: Day 1 — Foundations of AI Agents
[INFO] 15:40:32 — [FlashcardAgent] Generating flashcards for: Day 2 — Tools & Integrations
[INFO] 15:40:32 — [FlashcardAgent] Finished flashcards for: Day 2 — Tools & Integrations
[INFO] 15:40:32 — [FlashcardAgent] Generating flashcards for: Day 3 — Planning & Orchestration
[INFO] 15:40:32 — [FlashcardAgent] Finished flashcards for: Day 3 — Planning & Orchestration
[INFO] 15:40:32 

Created session id: 873ab1f8-90e9-4277-84de-03dfcfa6ff4b


[INFO] 15:40:32 — [SearchAgent] Finished search for: Day 1 — Foundations of AI Agents
[INFO] 15:40:32 — [SearchAgent] Finished search for: Day 2 — Tools & Integrations
[INFO] 15:40:32 — [SearchAgent] Finished search for: Day 3 — Planning & Orchestration
[INFO] 15:40:32 — [SearchAgent] Finished search for: Day 4 — Budgeting & Optimization
[INFO] 15:40:32 — [SearchAgent] Finished search for: Day 5 — Deployment & Observability



--- Plan Summary (topics) ---

Topic: Day 1 — Foundations of AI Agents
  Resources:
    - Day 1 — Foundations of AI Agents — Kuldeep's Playlist | https://www.youtube.com/@KulDeepLearns | A curated playlist covering Day 1 — Foundations of AI Agents....
    - Day 1 — Foundations of AI Agents — Deep dive article | https://example.com/deep | In-depth notes for Day 1 — Foundations of AI Agents....
  Flashcards: 4 created
  Quiz: 3 questions
  Informative queries:
    - prerequisites: Prerequisites for Day 1 — Foundations of AI Agents: basic Python, linear algebra (vectors/matrices), probability fundamentals, and comfort with reading technical blogs.
    - key_concepts: Key concepts for Day 1 — Foundations of AI Agents: core definitions, main algorithms, complexity trade-offs, common failure modes, and real-world examples.
    - practice_problems: Practice ideas for Day 1 — Foundations of AI Agents: implement a small example from scratch, solve 3 conceptual problems, and reproduce one resul

## 10. Long-running Agent (Pause & Resume Simulation)

This section simulates a long-running agent that:
- does work step-by-step,
- can be paused,
- can be resumed later.

This satisfies the "long-running operations" requirement.


In [10]:
class LongRunningAgent:
    def __init__(self):
        self.progress = 0
        self.paused = False

    def step(self):
        if self.paused:
            log.info("LongRunningAgent is paused. Cannot continue.")
            return

        self.progress += 10
        log.info(f"[LongRunningAgent] Progress: {self.progress}%")
        time.sleep(0.3)

        if self.progress >= 100:
            log.info("[LongRunningAgent] Task complete!")

    def pause(self):
        self.paused = True
        log.info("LongRunningAgent paused.")

    def resume(self):
        self.paused = False
        log.info("LongRunningAgent resumed.")


In [11]:
log.info("=== Demo: Long-running agent with pause/resume ===")

agent = LongRunningAgent()

# Run 3 steps
for _ in range(3):
    agent.step()

# Pause
agent.pause()

# Try running while paused
agent.step()

# Resume
agent.resume()

# Finish task
while agent.progress < 100:
    agent.step()


[INFO] 15:41:09 — === Demo: Long-running agent with pause/resume ===
[INFO] 15:41:09 — [LongRunningAgent] Progress: 10%
[INFO] 15:41:09 — [LongRunningAgent] Progress: 20%
[INFO] 15:41:10 — [LongRunningAgent] Progress: 30%
[INFO] 15:41:10 — LongRunningAgent paused.
[INFO] 15:41:10 — LongRunningAgent is paused. Cannot continue.
[INFO] 15:41:10 — LongRunningAgent resumed.
[INFO] 15:41:10 — [LongRunningAgent] Progress: 40%
[INFO] 15:41:10 — [LongRunningAgent] Progress: 50%
[INFO] 15:41:11 — [LongRunningAgent] Progress: 60%
[INFO] 15:41:11 — [LongRunningAgent] Progress: 70%
[INFO] 15:41:11 — [LongRunningAgent] Progress: 80%
[INFO] 15:41:11 — [LongRunningAgent] Progress: 90%
[INFO] 15:41:12 — [LongRunningAgent] Progress: 100%
[INFO] 15:41:12 — [LongRunningAgent] Task complete!


## 11. Results Interpretation

The system generated a complete study plan using multiple coordinated AI agents.

### ✅ 1. Content Resources (Search Agent)
For each topic, the `ContentSearchAgent` provided:
- A curated YouTube playlist (Dr. Abhishek’s AI Agent Intensive)
- A deep-dive article

This simulates how a real agent system would retrieve structured learning materials for a given topic.  
Logging messages show exactly when the search starts and ends for each topic.

---

### ✅ 2. Flashcards (Parallel Agent)
Flashcards were generated using `FlashcardAgent`, which runs **in parallel** with the content search using `ThreadPoolExecutor`.

For each topic:
- 4 flashcards are generated
- Each flashcard uses the LLM agent to produce a short explanation
- Logs indicate when flashcard generation starts and finishes

Parallel execution demonstrates **multi-agent performance optimization**.

---

### ✅ 3. Quizzes (LLM Agent Feature)
For each topic, the LLM agent generated 3 quiz questions.  
This simulates:
- Automatic assessment generation
- Basic knowledge checks
- AI-powered practice tools

---

### ✅ 4. Informative Queries (Prerequisites, Key Concepts, Practice)
For each topic, the assistant answers:
- **Prerequisites** → what the learner should know first
- **Key concepts** → the most important ideas
- **Practice problems** → how to apply the topic

This showcases how an LLM-powered agent can guide students with meta-learning support.

---

### ✅ 5. Sessions & Memory
The `InMemorySessionService` stores:
- `topics`
- `weekly_hours`
- `last_plan`

This proves the system can *remember* context and support personalized, ongoing learning for a user/session.

---

### ✅ 6. Observability (Logging)
Throughout the run, logging provides:
- Start/finish messages for search and flashcard agents
- Coordinator progress
- Long-running agent states (pause/resume, progress)

This adds **observability**, making the multi-agent system easier to debug and understand.

---

### ✅ 7. Long-running Agent (Pause & Resume)
The `LongRunningAgent` demo shows:
- A task that progresses in steps (10%, 20%, … 100%)
- Ability to **pause** mid-way
- Ability to **resume** and continue to completion

This simulates a **long-running operation** with pause/resume behavior, which is important for real-world agent workflows.

---

Overall, these results demonstrate:
- Multi-agent orchestration (search, flashcards, quizzes, guidance)
- Parallel and sequential execution
- Session-based personalization
- Observability via logging
- Long-running agent behavior

This meets and exceeds the three required features for the GenAI capstone.

## 12. Conclusion & Future Work

### ✔️ What This Project Achieves

This notebook implements an end-to-end **Smart AI Study Assistant** with:

- **Multi-agent system**
  - `ContentSearchAgent` for learning resources
  - `FlashcardAgent` for practice material
  - LLM-based agent (`MockLLMAgent`) for explanations, quizzes, and guidance
- **Parallel agents**
  - Search and flashcard generation run in parallel using `ThreadPoolExecutor`
- **Sequential agents**
  - Quizzes and informative queries are generated after resources and flashcards
- **Sessions & memory**
  - `InMemorySessionService` stores topics, weekly hours, and the last study plan
- **Observability**
  - Logging added to agents and demos to trace behavior
- **Long-running operations**
  - `LongRunningAgent` demonstrates a pausable and resumable workflow

The assistant can already:
- Generate resources for each topic
- Create flashcards
- Generate quiz questions
- Suggest prerequisites and practice ideas
- Track and reuse session-specific state
- Log its internal operations for better debugging and understanding
- Simulate a long-running agent flow with pause and resume

---

### 🚀 Future Enhancements

Several extensions can turn this foundation into a production-ready learning assistant:

#### **1. Integrate a Real LLM (Gemini / GPT)**
Replace `MockLLMAgent` with a real LLM API, using prompt engineering and safety checks for:
- explanations
- quizzes
- follow-up questions

#### **2. Add Long-term Memory (Memory Bank)**
Persist:
- user progress
- past plans
- mastered topics

across sessions using a database or vector store.

#### **3. Add Agent Evaluation**
- Measure quiz performance
- Track time spent per topic
- Estimate mastery level and adapt difficulty

#### **4. Context Engineering**
- Summarize long histories into compact context
- Feed only the most relevant pieces to the LLM

#### **5. API / UI Layer**
- Serve the assistant via FastAPI
- Build a simple frontend (Streamlit/React) for interactive use

---

### 🏁 Final Thoughts

This project demonstrates how multiple core ideas from the course can come together:

- Multi-agent collaboration  
- LLM-powered guidance  
- Parallel and sequential workflows  
- Session and state management  
- Observability through logging  
- Long-running operation simulation  

It provides a solid base that can be extended into a more advanced, real-world AI agent system for education.
